Copyright (c) 2021, salesforce.com, inc. \
All rights reserved. \
SPDX-License-Identifier: BSD-3-Clause \
For full license text, see the LICENSE file in the repo root or https://opensource.org/licenses/BSD-3-Clause

**Try this notebook on [Colab](http://colab.research.google.com/github/salesforce/warp-drive/blob/master/tutorials/tutorial-2-warp_drive_sampler.ipynb)!**

# ⚠️ PLEASE NOTE:
This notebook runs on a GPU runtime.\
If running on Colab, choose Runtime > Change runtime type from the menu, then select `GPU` in the 'Hardware accelerator' dropdown menu.

In [1]:
import torch

assert torch.cuda.device_count() > 0, "This notebook needs a GPU to run!"

# Welcome to WarpDrive!

This is the second tutorial on WarpDrive, a PyCUDA-based framework for extremely parallelized multi-agent reinforcement learning (RL) on a single graphics processing unit (GPU). At this stage, we assume you have read our [first tutorial](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-1-warp_drive_basics.ipynb) on WarpDrive basics.

In this tutorial, we describe **CUDASampler**, a lightweight and fast action sampler based on the policy distribution across several RL agents and environment replicas. `CUDASampler` utilizes the GPU to parallelize operations to efficiently sample a large number of actions in parallel. 

Notably:

1. It reads the distribution on the GPU through Pytorch and samples actions exclusively at the GPU. There is no data transfer. 
2. It maximizes parallelism down to the individual thread level, i.e., each agent at each environment has its own random seed and independent random sampling process. 
3. It runs much faster than most GPU samplers. For example, it is significantly faster than Pytorch.

# Dependencies

You can install the warp_drive package using

- the pip package manager, OR
- by cloning the warp_drive package and installing the requirements.

On Colab, we will do the latter.

In [2]:
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    ! git clone https://github.com/salesforce/warp-drive.git
    % cd warp-drive
    ! pip install -e .
else:
    ! pip install -U rl_warp_drive

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
from warp_drive.managers.function_manager import CUDAFunctionManager, CUDASampler
from warp_drive.managers.data_manager import CUDADataManager
from warp_drive.utils.constants import Constants
from warp_drive.utils.data_feed import DataFeed
from warp_drive.utils.common import get_project_root

_MAIN_FILEPATH = f"{get_project_root()}/warp_drive/cuda_includes"
_CUBIN_FILEPATH = f"{get_project_root()}/warp_drive/cuda_bin"
_ACTIONS = Constants.ACTIONS

In [4]:
# Set logger level e.g., DEBUG, INFO, WARNING, ERROR
import logging

logging.getLogger().setLevel(logging.INFO)

# Initialize CUDASampler

We first initialize the **CUDADataManager** and **CUDAFunctionManager**. To illustrate the sampler, we first load a pre-compiled binary file called "test_build.cubin". Note that these low-level managers and modules will be hidden and called automatically by WarpDrive in any end-to-end training and simulation. In this and the next tutorials, we want to show how a few fundamental modules work and their performance, that is why some low-level APIs are called.

In [5]:
cuda_data_manager = CUDADataManager(num_agents=5, episode_length=10, num_envs=2)
cuda_function_manager = CUDAFunctionManager(
    num_agents=cuda_data_manager.meta_info("n_agents"),
    num_envs=cuda_data_manager.meta_info("n_envs"),
)

main_example_file = f"{_MAIN_FILEPATH}/test_build.cu"
bin_example_file = f"{_CUBIN_FILEPATH}/test_build.fatbin"

INFO:root:
Pushing data to device...
INFO:root:- _log_mask_                                                                      : dtype=int32     , shape=(11,)
INFO:root:
Pushing data to device...
INFO:root:- _done_                                                                          : dtype=int32     , shape=(2,)
INFO:root:
Pushing data to device...
INFO:root:- _timestep_                                                                      : dtype=int32     , shape=(2,)


This binary is compiled with inclusion of auxiliary files in `warp_drive/cuda_includes/core` which includes several CUDA core services provided by WarpDrive. These include the backend source code for `CUDASampleController`. 

To make "test_build.fatbin" available, we compiled this test cubin by calling `_compile()` from `CUDAFunctionManager`.
For this notebook demonstration, in the bin folder, we have already provided a pre-compiled binary but we suggest that you still execute the cell below to re-compile it to avoid possilble binary incompatible issues across different platforms. (`_compile()` is a low-level API, user will not need call those internal APIs directly for any WarpDrive end-to-end simulation and training.)

In [6]:
cuda_function_manager._compile(main_file=main_example_file, 
                               cubin_file=bin_example_file)

INFO:root:Successfully mkdir the binary folder /home/jupyter/.local/lib/python3.8/site-packages/warp_drive/cuda_bin


/home/jupyter/.local/lib/python3.8/site-packages/warp_drive/cuda_includes/../../example_envs/tag_gridworld/tag_gridworld_step.cu(151): warning: invalid narrowing conversion from "unsigned int" to "int"



INFO:root:Running cmd: nvcc --fatbin -arch=sm_70 /home/jupyter/.local/lib/python3.8/site-packages/warp_drive/cuda_includes/test_build.cu -o /home/jupyter/.local/lib/python3.8/site-packages/warp_drive/cuda_bin/test_build.fatbin
INFO:root:Successfully build the cubin_file from /home/jupyter/.local/lib/python3.8/site-packages/warp_drive/cuda_includes/test_build.cu to /home/jupyter/.local/lib/python3.8/site-packages/warp_drive/cuda_bin/test_build.fatbin


Finally, we initialize **CUDASampler** and assign the random seed. `CUDASampler` keeps independent randomness across all threads and blocks. Notice that `CUDASampler` requires `CUDAFunctionManager` because `CUDAFunctionManager` manages all the CUDA function pointers including to the sampler. Also notice this test binary uses 2 environment replicas and 5 agents. 

In [7]:
cuda_function_manager.load_cuda_from_binary_file(
    bin_example_file, default_functions_included=True
)
cuda_sampler = CUDASampler(function_manager=cuda_function_manager)
cuda_sampler.init_random(seed=None)

INFO:root:Successfully load the cubin_file from /home/jupyter/.local/lib/python3.8/site-packages/warp_drive/cuda_bin/test_build.fatbin
INFO:root:starting to load the cuda kernel function: reset_log_mask from the CUDA module 
INFO:root:finished loading the cuda kernel function: reset_log_mask from the CUDA module, 
INFO:root:starting to load the cuda kernel function: update_log_mask from the CUDA module 
INFO:root:finished loading the cuda kernel function: update_log_mask from the CUDA module, 
INFO:root:starting to load the cuda kernel function: log_one_step_in_float from the CUDA module 
INFO:root:finished loading the cuda kernel function: log_one_step_in_float from the CUDA module, 
INFO:root:starting to load the cuda kernel function: log_one_step_in_int from the CUDA module 
INFO:root:finished loading the cuda kernel function: log_one_step_in_int from the CUDA module, 
INFO:root:starting to load the cuda kernel function: reset_in_float_when_done_2d from the CUDA module 
INFO:root:fi

# Sampling

## Actions Placeholder

Now, we feed the **actions_a** placeholder into the GPU. It has the shape `(n_envs=2, n_agents=5)` as expected. Also we make it accessible by Pytorch, because during RL training, actions will be fed into the Pytorch trainer directly.

In [8]:
data_feed = DataFeed()
data_feed.add_data(name=f"{_ACTIONS}_a", data=[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]])
cuda_data_manager.push_data_to_device(data_feed, torch_accessible=True)
assert cuda_data_manager.is_data_on_device_via_torch(f"{_ACTIONS}_a")

INFO:root:
Pushing data to device...
INFO:root:- sampled_actions_a                                                               : dtype=int32     , shape=(2, 5)


## Action Sampled Distribution

We define an action **distribution** here. During training, this distribution would be provided by the policy model implemented in Pytorch. The distribution has the shape `(n_envs, n_agents, **n_actions**)`. The last dimension `n_actions` defines the size of the action space for a particular *discrete* action. For example, if we have up, down, left, right and no-ops, `n_actions=5`.

**n_actions** needs to be registered by the sampler so the sampler is able to pre-allocate a global memory space in GPU to speed up action sampling. This can be done by calling `sampler.register_actions()`.

In this tutorial, we check if our sampled action distribution follows the given distribution. For example, the distribution [0.333, 0.333, 0.333] below suggests the 1st agent has 3 possible actions and each of them have equal probability.

In [9]:
cuda_sampler.register_actions(
    cuda_data_manager, action_name=f"{_ACTIONS}_a", num_actions=3
)

distribution = np.array(
    [
        [
            [0.333, 0.333, 0.333],
            [0.2, 0.5, 0.3],
            [0.95, 0.02, 0.03],
            [0.02, 0.95, 0.03],
            [0.02, 0.03, 0.95],
        ],
        [
            [0.1, 0.7, 0.2],
            [0.7, 0.2, 0.1],
            [0.5, 0.5, 0.0],
            [0.0, 0.5, 0.5],
            [0.5, 0.0, 0.5],
        ],
    ]
)
distribution = torch.from_numpy(distribution).float().cuda()

INFO:root:
Pushing data to device...
INFO:root:- sampled_actions_a_cum_distr                                                     : dtype=float32   , shape=(2, 5, 3)


In [10]:
# Run 10000 times to collect statistics
actions_batch = torch.from_numpy(np.empty((10000, 2, 5), dtype=np.int32)).cuda()

for i in range(10000):
    cuda_sampler.sample(cuda_data_manager, distribution, action_name=f"{_ACTIONS}_a")
    actions_batch[i] = cuda_data_manager.data_on_device_via_torch(f"{_ACTIONS}_a")
actions_batch_host = actions_batch.cpu().numpy()

In [11]:
actions_env_0 = actions_batch_host[:, 0]
actions_env_1 = actions_batch_host[:, 1]

In [12]:
print(
    "Sampled actions distribution versus the given distribution (in bracket) for env 0: \n"
)
for agent_id in range(5):
    print(
        f"Sampled action distribution for agent_id: {agent_id}:\n"
        f"{(actions_env_0[:, agent_id] == 0).sum() / 10000.0}({distribution[0, agent_id, 0]}), \n"
        f"{(actions_env_0[:, agent_id] == 1).sum() / 10000.0}({distribution[0, agent_id, 1]}), \n"
        f"{(actions_env_0[:, agent_id] == 2).sum() / 10000.0}({distribution[0, agent_id, 2]})  \n"
    )

Sampled actions distribution versus the given distribution (in bracket) for env 0: 

Sampled action distribution for agent_id: 0:
0.3368(0.3330000042915344), 
0.3252(0.3330000042915344), 
0.338(0.3330000042915344)  

Sampled action distribution for agent_id: 1:
0.2018(0.20000000298023224), 
0.4954(0.5), 
0.3028(0.30000001192092896)  

Sampled action distribution for agent_id: 2:
0.9528(0.949999988079071), 
0.0193(0.019999999552965164), 
0.0279(0.029999999329447746)  

Sampled action distribution for agent_id: 3:
0.0183(0.019999999552965164), 
0.9501(0.949999988079071), 
0.0316(0.029999999329447746)  

Sampled action distribution for agent_id: 4:
0.0208(0.019999999552965164), 
0.032(0.029999999329447746), 
0.9472(0.949999988079071)  



In [13]:
print(
    "Sampled actions distribution versus the given distribution (in bracket) for env 1: "
)

for agent_id in range(5):
    print(
        f"Sampled action distribution for agent_id: {agent_id}:\n"
        f"{(actions_env_1[:, agent_id] == 0).sum() / 10000.0}({distribution[1, agent_id, 0]}), \n"
        f"{(actions_env_1[:, agent_id] == 1).sum() / 10000.0}({distribution[1, agent_id, 1]}), \n"
        f"{(actions_env_1[:, agent_id] == 2).sum() / 10000.0}({distribution[1, agent_id, 2]})  \n"
    )

Sampled actions distribution versus the given distribution (in bracket) for env 1: 
Sampled action distribution for agent_id: 0:
0.0998(0.10000000149011612), 
0.7032(0.699999988079071), 
0.197(0.20000000298023224)  

Sampled action distribution for agent_id: 1:
0.6898(0.699999988079071), 
0.209(0.20000000298023224), 
0.1012(0.10000000149011612)  

Sampled action distribution for agent_id: 2:
0.4943(0.5), 
0.5057(0.5), 
0.0(0.0)  

Sampled action distribution for agent_id: 3:
0.0(0.0), 
0.4979(0.5), 
0.5021(0.5)  

Sampled action distribution for agent_id: 4:
0.4923(0.5), 
0.0(0.0), 
0.5077(0.5)  



## Action Randomness Across Threads

Another important validation is whether the sampler provides independent randomness across different agents and environment replicas. Given the same policy model for all the agents and environment replicas, we can check if the sampled actions are independently distributed. 

Here, we assign all agents across all envs the same distribution [0.25, 0.25, 0.25, 0.25]. It is equivalent to an uniform action distribution among all actions [0,1,2,3], across 5 agents and 2 envs. Then we check the standard deviation across the agents.

In [14]:
data_feed = DataFeed()
data_feed.add_data(name=f"{_ACTIONS}_b", data=[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]])
cuda_data_manager.push_data_to_device(data_feed, torch_accessible=True)
assert cuda_data_manager.is_data_on_device_via_torch(f"{_ACTIONS}_b")

INFO:root:
Pushing data to device...
INFO:root:- sampled_actions_b                                                               : dtype=int32     , shape=(2, 5)


In [15]:
cuda_sampler.register_actions(
    cuda_data_manager, action_name=f"{_ACTIONS}_b", num_actions=4
)

INFO:root:
Pushing data to device...
INFO:root:- sampled_actions_b_cum_distr                                                     : dtype=float32   , shape=(2, 5, 4)


In [16]:
distribution = np.array(
    [
        [
            [0.25, 0.25, 0.25, 0.25],
            [0.25, 0.25, 0.25, 0.25],
            [0.25, 0.25, 0.25, 0.25],
            [0.25, 0.25, 0.25, 0.25],
            [0.25, 0.25, 0.25, 0.25],
        ],
        [
            [0.25, 0.25, 0.25, 0.25],
            [0.25, 0.25, 0.25, 0.25],
            [0.25, 0.25, 0.25, 0.25],
            [0.25, 0.25, 0.25, 0.25],
            [0.25, 0.25, 0.25, 0.25],
        ],
    ]
)
distribution = torch.from_numpy(distribution).float().cuda()

In [17]:
# Run 10000 times to collect statistics.
actions_batch = torch.from_numpy(np.empty((10000, 2, 5), dtype=np.int32)).cuda()

for i in range(10000):
    cuda_sampler.sample(cuda_data_manager, distribution, action_name=f"{_ACTIONS}_b")
    actions_batch[i] = cuda_data_manager.data_on_device_via_torch(f"{_ACTIONS}_b")
actions_batch_host = actions_batch.cpu().numpy()

In [18]:
actions_batch_host

array([[[0, 1, 0, 1, 1],
        [1, 0, 2, 3, 1]],

       [[1, 3, 0, 3, 1],
        [3, 3, 3, 3, 1]],

       [[0, 1, 2, 2, 2],
        [3, 2, 1, 0, 1]],

       ...,

       [[3, 0, 2, 2, 2],
        [1, 1, 1, 0, 1]],

       [[1, 1, 0, 1, 0],
        [1, 0, 2, 3, 0]],

       [[2, 2, 0, 1, 0],
        [1, 2, 2, 2, 3]]], dtype=int32)

In [19]:
actions_batch_host.std(axis=2).mean(axis=0)

array([0.96386563, 0.96353072])

To check the independence of randomness among all threads, we can compare it with a Numpy implementation. Here we use `numpy.choice(4, 5)` to repeat the same process for an uniform action distribution among all actions [0,1,2,3], 5 agents and 2 envs. We should see that the variation of Numpy output is very close to our sampler.

In [20]:
actions_batch_numpy = np.empty((10000, 2, 5), dtype=np.int32)
for i in range(10000):
    actions_batch_numpy[i, 0, :] = np.random.choice(4, 5)
    actions_batch_numpy[i, 1, :] = np.random.choice(4, 5)
actions_batch_numpy.std(axis=2).mean(axis=0)

array([0.96419519, 0.96432735])

## Running Speed

The total time for sampling includes receiving a new distribution and using this to sample.
Comparing our sampler with [torch.Categorical sampler](https://pytorch.org/docs/stable/distributions.html), 
we reach **7-8X** speed up for the distribution above. 

*Note: our sampler runs in parallel across threads, so this speed-up is almost constant when scaling up the number of agents or environment replicas, i.e., increasing the number of used threads.*

In [21]:
from torch.distributions import Categorical

In [22]:
distribution = np.array(
    [
        [
            [0.333, 0.333, 0.333],
            [0.2, 0.5, 0.3],
            [0.95, 0.02, 0.03],
            [0.02, 0.95, 0.03],
            [0.02, 0.03, 0.95],
        ],
        [
            [0.1, 0.7, 0.2],
            [0.7, 0.2, 0.1],
            [0.5, 0.5, 0.0],
            [0.0, 0.5, 0.5],
            [0.5, 0.0, 0.5],
        ],
    ]
)
distribution = torch.from_numpy(distribution).float().cuda()

In [23]:
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

start_event.record()
for _ in range(1000):
    cuda_sampler.sample(cuda_data_manager, distribution, action_name=f"{_ACTIONS}_a")
end_event.record()
torch.cuda.synchronize()
print(f"time elapsed: {start_event.elapsed_time(end_event)} ms")

time elapsed: 26.30963134765625 ms


In [24]:
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

start_event.record()
for _ in range(1000):
    Categorical(distribution).sample()
end_event.record()
torch.cuda.synchronize()
print(f"time elapsed: {start_event.elapsed_time(end_event)} ms")

time elapsed: 320.6543273925781 ms


# Learn More and Explore our Tutorials!

Next, we suggest you check out our advanced [tutorial](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-3-warp_drive_reset_and_log.ipynb) on WarpDrive's reset and log controller!

For your reference, all our tutorials are here:
1. [WarpDrive basics](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-1-warp_drive_basics.ipynb)
2. [WarpDrive sampler](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-2-warp_drive_sampler.ipynb)
3. [WarpDrive reset and log](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-3-warp_drive_reset_and_log.ipynb)
4. [Creating custom environments](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-4-create_custom_environments.md)
5. [Training with WarpDrive](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-5-training_with_warp_drive.ipynb)
6. [Scaling Up training with WarpDrive](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-6-scaling_up_training_with_warp_drive.md)
7. [Training with WarpDrive + Pytorch Lightning](https://github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-7-training_with_warp_drive_and_pytorch_lightning.ipynb)